In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import splitfolders
import cv2
import os

In [2]:
input_folder = '/Users/Gordon Freeman/ml-notebooks/Agricultural-crops'
output_folder = '/Users/Gordon Freeman/ml-notebooks/ImageRecognition' # save our file

split_ratio = (0.8, 0.1, 0.1) # 80% train, 10% validation, 10% test
splitfolders.ratio(
    input_folder, 
    output=output_folder, 
    seed=500, # random number generator, ensures the split is reproducible so running with the same seed means the same split
              # order types of ml models use `random-state`
    ratio=split_ratio, 
    group_prefix=None
)

In [3]:
img_size = (224, 224) # resize the images to 224x224 pixels, this is a common size usef for deep learning
batch_size = 32       # models weight is updated after it processes 32 images

# data augmentation for the training data to expand the dataset with transformed versions, improves model generalization
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # resnet50 pre trained model
    rotation_range=20,                       # randomly rotate the image by up to 20 degrees
    width_shift_range=0.2,                   # randomly shift the image horizontally left/right by up to 20% of the width
    height_shift_range=0.2,                  # randomly shift the image vertically up/down by up to 20% of the height
    shear_range=0.2,                         # random shear transformations up to 20%
    zoom_range=0.2,                          # randomly zooms into the image up to 20%
    horizontal_flip=True,                    # randomly flips the image
    fill_mode='nearest'                      # when the image is rotated/shifted and a new pixel needs to be filled in, the nearest is used
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # data augmentation for test data (only rescaling)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # data augmentation for validation data (only rescaling)

In [4]:
train_dir = os.path.join(output_folder, 'train')
test_dir = os.path.join(output_folder, 'test')
val_dir = os.path.join(output_folder, 'val')

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical' # type of label array to be returned, categorical means the labels will be one hot encoded, useful for multiclass classification
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 652 images belonging to 30 classes.
Found 105 images belonging to 30 classes.
Found 105 images belonging to 30 classes.


In [5]:
from keras.applications.resnet import ResNet50 # Convolutional Neural Networks (CNN) that has been pre-trained on images
base_model = ResNet50(
    weights='imagenet',                       # use weights of the model that has been pre-trained
    include_top=False,                        # dont include the fully connected layers at the top of the network
                                              # the top refers to the classification layers that are normally at the end of our network
                                              # by excluding this we can add our own custom classification layers suitable for our problem
    input_shape=(img_size[0], img_size[1], 3) # shape of input images, they are expected 224 by 224px with 3 colour channels RGB (red/green/blue)
)

base_model.trainable=False # freeze convolutional base, meaning the weights of these layers will not be updated during training
                           # done to preserve the pre-trained weights and only train the newly added classification layers
                           # freezing the base model helps to leverage the features learnt from pre training without altering them

In [6]:
model = models.Sequential([
    base_model,                            # pre trained ResNet50 model
    layers.GlobalAveragePooling2D(),       # used to replace fully connected layers in CNNs to reduce overfitting and the number of parameters
    layers.Dense(128, activation='relu'),  # fully connected dense layer with 128 neurons, relu (rectified linear unit) is the activation function
                                           # relu introduces nonlinearity enabling the model to learn more complex representations
    layers.Dropout(0.5),                   # randomly sets 50% of its input units to zero during each update
                                           # this also helps to prevent overfitting
    layers.Dense(30, activation='softmax') # another fully connected layer with 30 neurons
                                           # softmax activation function transforms raw output scores (logits) into a probability distribution
                                           # 
])

In [8]:
model.compile(
    optimizer='adam',                # updates the models weight during training to minimize the loss function
                                     # `adam` (adaotive moment estimation) is an advanced gradient descent algorithm that adjusts the learning rate for each parameter 
                                     # this done based on estimates for lower order moments
                                     # other optimizers exist but `adam` is widely used because its computationally efficient and requires less memory

    loss='categorical_crossentropy', # loss functions in ml measure how well the models prediction matches the true actual values
                                     # during training the optimizer tries to minimize the loss
                                     # other loss functions like `Mean Squared Error (MSE)` exist

    metrics=['accuracy']             # metrics evaluate the performance of the model
)

In [16]:
model.fit(
    train_data,
    epochs=100,                # epoch is one complete pass though the training data set, so 100 means do it 100 times (all batches)
                               # 100 is very common in examples (should take about 30 minutes on a standard desktop machine)
                               # the idea is with each epoch of time, the `accuracy` goes up and the `loss` goes down
    validation_data=valid_data
)

Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.4839 - loss: 1.8064 - val_accuracy: 0.5619 - val_loss: 1.4423
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.5370 - loss: 1.5586 - val_accuracy: 0.6381 - val_loss: 1.3040
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.5619 - loss: 1.4978 - val_accuracy: 0.6571 - val_loss: 1.2561
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5919 - loss: 1.4334 - val_accuracy: 0.6762 - val_loss: 1.1420
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5974 - loss: 1.2830 - val_accuracy: 0.6667 - val_loss: 1.0668
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6544 - loss: 1.1018 - val_accuracy: 0.7048 - val_loss: 1.0384
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.6968 - loss: 1.1063 - val_accuracy: 0.7238 - val_loss: 0.9744
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.6849 - loss: 1.1422 - val_accuracy: 0.7143 - v

In [17]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8078 - loss: 0.8704
Test Accuracy: 81.90%


In [13]:
class_names = {
    0: 'Cherry',
    1: 'Coffee-plant',
    2: 'Cucumber',
    3: 'Fox_nut(Makhana)',
    4: 'Lemon',
    5: 'Olive-tree',
    6: 'Pearl_millet(bajra)',
    7: 'Tobacco-plant',
    8: 'almond',
    9: 'banana',
    10: 'cardamom',
    11: 'chilli',
    12: 'clove',
    13: 'coconut',
    14: 'cotton',
    15: 'gram',
    16: 'jowar',
    17: 'jute',
    18: 'maize',
    19: 'mustard-oil',
    20: 'papaya',
    21: 'pineapple',
    22: 'rice',
    23: 'soyabean',
    24: 'sugarcane',
    25: 'sunflower',
    26: 'tea',
    27: 'tomato',
    28: 'vigna-radiati(Mung)',
    29: 'wheat'
}

In [14]:
def predict_img(image, model):
    test_img=cv2.imread(image)                # read the image from the specified file path as an array
    test_img=cv2.resize(test_img, (224,224))  # resize to 224 by 224px to match the size the model was trained on
    test_img=np.expand_dims(test_img, axis=0) # numpy function to add extra dimensions to the image array
    result=model.predict(test_img)            # use trained model to make prediction
    r=np.argmax(result)                       # returns the index of the maxium value in the result array, 
                                              # this should correspond to the class with the highest probability
    print(class_names[r])

In [19]:
predict_img(
    '/Users/Gordon Freeman/ml-notebooks/Agricultural-crops/Pearl_millet(bajra)/image (50).jpg',
    model
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Pearl_millet(bajra)


In [20]:
model.save('Agricultural-crops.keras')